# Imports

In [1]:
import pandas as pd

from helpers import get_meta_info, read_specific_data, get_letter, columns_with_value_contains, write_to_db, sql_types_metadata

# Definitions

In [2]:
file_name = 'JA 2021 MG_2.xls'
sheet_name = "Pasettistraße 1OG"
file_path, _, _ = get_meta_info(file_name, 1, 1)

# Code

In [3]:
# vorgehen wie in 04_get_metadaten_schliesszeiten

# read the whole sheet
df = pd.read_excel(file_path, sheet_name=sheet_name, nrows=1000)

# use simple regex to find the starting start_row. The "key_word" indicates the starting point. 
key_word = 'D. ÖFFNUNGSZEITEN'


mask = df.apply(lambda row: row.astype(str).str.contains(key_word, case=False).any(), axis=1)
rows_with_key_word = df[mask]


start_row = rows_with_key_word.index[0] + 2 # due to header columns etc.
end_row = start_row + 14

# find the starting columns, returns column name (based on dataframe, and column where the data is located)
columns_containing = columns_with_value_contains(rows_with_key_word, key_word)

alle_zeiten = pd.DataFrame()
ebene_1  = "D. ÖFFNUNGSZEITEN 3)"
ebene_2  = "n. a."
ebene_3  = "n. a."
erlaeuterung = 'n. a.'

for _, idx, in columns_containing:
    use_col = get_letter(idx+2) + ", Q"
    data = read_specific_data(file_name, start_row, end_row, sheet_name, use_col, ebene_1, ebene_2, ebene_3)
    data = data.iloc[1:]
    alle_zeiten = pd.concat([alle_zeiten, data], ignore_index=True)

alle_zeiten.columns.values[0] = "Name_Eintrag"
alle_zeiten.columns.values[1] = "Eintrag"
alle_zeiten['Erlaeuterung'] = erlaeuterung

table_name = 'Metadaten'
alle_zeiten['Eintrag'] = alle_zeiten['Eintrag'].astype('object')
write_to_db(table_name, alle_zeiten, file_name, sql_types = sql_types_metadata)

file_path: '..\02_data\01_input\nok\JA 2021 MG_2.xls', skip_to: 44, amount_of_rows: 15
file_path: '..\02_data\01_input\nok\JA 2021 MG_2.xls', skip_to: 2, amount_of_rows: 2
file_path: '..\02_data\01_input\nok\JA 2021 MG_2.xls', skip_to: 1, amount_of_rows: 1
Data inserted successfully.
